# Exploração e análise de dados de crédito com SQL

Os dados representam informações de clientes de um banco e contêm as seguintes colunas:

* idade = idade do cliente
* sexo = sexo do cliente (M ou F)
* dependentes = número de dependentes do cliente
* escolaridade = nível de escolaridade dos clientes
* estado_civil = estado civil do cliente
* salario_anual = faixa salarial do cliente
* tipo_cartao = tipo de cartão do cliente
* qtd_produtos = quantidade de produtos comprados nos últimos 12 meses
* iteracoes_12m = quantidade de iterações feitas nos últimos 12 meses
* meses_inativo_12m = quantidade de meses que o cliente ficou inativo
* limite_credito = limite de crédito do cliente
* valor_transacoes_12m = valor das transações nos últimos 12 meses
* qtd_transacoes_12m = quantidade de transações dos últimos 12 meses

A tabela de SQL foi criada usando o **AWS Athena** através de um arquivo colocado em um **Bucket S3**. 

## 1. Limpeza
Primeiramente, precisamos limpar o dataset colocando os valores decimais num formato legível pelo AWS Athena. Ou seja, vírgulas são substituídas por pontos. Além disso, de cerca de 10 mil linhas reduzimos a 2500 para menor custo de computação.

In [1]:
import pandas as pd

df = pd.read_csv('../input/credit-data/credito.csv', sep=',')

# Limpeza de dados
cols = ['limite_credito', 'valor_transacoes_12m']

for col in cols:
  df[col] = df[col].apply(lambda x: x.replace('.', '').replace(',', '.'))

# Removendo colunas que não são úteis na análise
df = df.drop(['id', 'default', 'meses_de_relacionamento'], axis=1)

# Limitando a quantidade de dados a 2500 linhas e sem cabeçalho
df = df.iloc[0:2500]
df.to_csv('../working/credito_clean.csv', sep=',', header=False, index=False)

## 2. Criação da tabela e descrição

No AWS S3 é criado um novo bucket s3://bucket-projeto-sql/ onde é colocado o arquivo credito_clean.csv. Na AWS Athena é criada a tabela 'credito' a partir deste arquivo csv.

```
CREATE EXTERNAL TABLE credito(
  idade INT,
  sexo STRING,
  dependentes INT,
  escolaridade STRING,
  estado_civil STRING,
  salario_anual STRING,
  tipo_cartao STRING,
  qtd_produtos BIGINT,
  iteracoes_12m INT,
  meses_inativo_12m INT,
  limite_credito FLOAT,
  valor_transacoes_12m FLOAT,
  qtd_transacoes_12m INT
)
ROW FORMAT SERDE 'org.apache.hadoop.hive.serde2.lazy.LazySimpleSerDe'
WITH SERDEPROPERTIES ('serialization.format' = ',', 'field.delim' = ',')
LOCATION 's3://bucket-projeto-sql/'
TBLPROPERTIES('has_encrypted_data' = 'false');
```

Através da query SQL

```
DESCRIBE credito;
```

é possível ver todas as colunas da tabela formada e seus tipos de dados.

<img src='https://github.com/mateus-miguel/projeto-credito-sql/blob/main/imagens/describe_sql.png?raw=true' style='display:block;float:none;margin-left:auto;margin-right:auto;width:30%'/>

## 3. Análise Exploratória de Dados

### 3.1 - Categorias de dados

A query

```
SELECT DISTINCT escolaridade
FROM credito;
```

mostra que existem **cinco tipos de escolaridade** na tabela: sem educação formal, ensino medio, graduação, mestrado e doutorado. O tipo 'na' apenas indica erro na hora de processar a informação num cadastro, ou dados faltantes. Logo, precisa ser filtrada em outras consultas.

<img src='https://github.com/mateus-miguel/projeto-credito-sql/blob/main/imagens/escolaridade_sql.png?raw=true' style='display:block;float:none;margin-left:auto;margin-right:auto;width:30%'/>

Com outra query

```
SELECT DISTINCT estado_civil
FROM credito;
```

<img src='https://github.com/mateus-miguel/projeto-credito-sql/blob/main/imagens/estado_civil_sql.png?raw=true' style='display:block;float:none;margin-left:auto;margin-right:auto;width:25%'/>

Consegue-se ver que existem **três tipos de estados civis** disponíveis: solteiro, casado ou divorciado e 'na' dos dados faltantes.

Já pela consulta

```
SELECT DISTINCT salario_anual
FROM credito;
```

<img src='https://github.com/mateus-miguel/projeto-credito-sql/blob/main/imagens/salario_anual_sql.png?raw=true' style='display:block;float:none;margin-left:auto;margin-right:auto;width:25%'/>

Para salário anual temos cerca de **cinco faixas salariais**: menos que \\$40K, \\$40K - \\$60K, \\$60K - \\$80K, \\$80K - \\$120K, \\$120K + e 'na'.

### 3.2 - Agregações

Uma forma de organizar as informações é agrupar os dados através destas categorias. Com a query

```
SELECT COUNT(*) AS qtde, sexo
FROM credito
GROUP BY sexo;
```

Nota-se que a maioria dos clientes na base de dados são homens, o que influencia resultados de outras queries.

<img src='https://github.com/mateus-miguel/projeto-credito-sql/blob/main/imagens/sexo_groupby.png?raw=true' style='display:block;float:none;margin-left:auto;margin-right:auto;width:80%'/>

O gráfico de pizza abaixo destaca os dois grupos.

<img src='https://github.com/mateus-miguel/projeto-credito-sql/blob/main/imagens/sexo_groupby_pie.png?raw=true' style='display:block;float:none;margin-left:auto;margin-right:auto;width:60%'/>

**Qual é a distribuição dos clientes pelas faixas salariais?**

```
SELECT COUNT(*) AS qtde, salario_anual
FROM credito
GROUP BY salario_anual
ORDER BY qtde ASC;
```

Conseguimos ver que o maior grupo de pessoas está na faixa salarial de 'menos que \\$40K' e que uma minoria possui salário na faixa de '\\$120K +'.

<img src='https://github.com/mateus-miguel/projeto-credito-sql/blob/main/imagens/salario_anual_groupby.png?raw=true' style='display:block;float:none;margin-left:auto;margin-right:auto;width:50%'/>

O gráfico de pizza a seguir ilustra bem esses grupos salariais.

<img src='https://github.com/mateus-miguel/projeto-credito-sql/blob/main/imagens/salario_anual_groupby_pie.png?raw=true' style='display:block;float:none;margin-left:auto;margin-right:auto;width:60%'/>

**Com base no sexo, quais os tipos de cartão mais utilizados?**

Outro tipo de agregação útil é contar a quantidade de clientes por sexo utilizando determinado tipo de cartão. A query para isto é

```
SELECT COUNT(*) AS qtde, sexo, tipo_cartao
FROM credito
GROUP BY sexo, tipo_cartao
ORDER BY qtde DESC;
```

<img src='https://github.com/mateus-miguel/projeto-credito-sql/blob/main/imagens/sexo_tipo_cartao_groupby.png?raw=true' style='display:block;float:none;margin-left:auto;margin-right:auto;width:60%'/>

A partir desta consulta vemos que a maioria dos clientes usa o cartão 'blue', e que os homens são a maior parte dos usuários de todos os tipos de cartão. Porém, isto também pode se dever ao conjunto de dados conter mais homens do que mulheres. O gráfico de barra a seguir deixa esta diferença mais evidente.

<img src='https://github.com/mateus-miguel/projeto-credito-sql/blob/main/imagens/sexo_tipo_cartao_groupby_barra.png?raw=true' style='display:block;float:none;margin-left:auto;margin-right:auto;width:70%'/>

**E se quisermos focar estratégias de marketing nos clientes, qual a idade deles?**

```
SELECT AVG(idade) AS media_idade, MIN(idade) AS min_idade, MAX(idade) AS max_idade, sexo
FROM credito
GROUP BY sexo;
```

<img src='https://github.com/mateus-miguel/projeto-credito-sql/blob/main/imagens/idade_sexo_groupby.png?raw=true' style='display:block;float:none;margin-left:auto;margin-right:auto;width:80%'/>

Com as funções estatísticas de agregação vemos que os homens apresentam idade máxima maior, mas tanto homens quanto mulheres têm mesma idade média na tabela.

<img src='https://github.com/mateus-miguel/projeto-credito-sql/blob/main/imagens/idade_sexo_groupby_barra.png?raw=true' style='display:block;float:none;margin-left:auto;margin-right:auto;width:70%'/>

**Qual a quantidade de clientes por cada estado civil?**

```
SELECT COUNT(*) AS qtd, estado_civil
FROM credito
GROUP BY estado_civil
ORDER BY qtd DESC;
```

<img src='https://github.com/mateus-miguel/projeto-credito-sql/blob/main/imagens/estado_civil_groupby.png?raw=true' style='display:block;float:none;margin-left:auto;margin-right:auto;width:80%'/>

Vemos que a maioria dos clientes do banco é casado, seguidos dos solteiros e a minoria são divorciados.

<img src='https://github.com/mateus-miguel/projeto-credito-sql/blob/main/imagens/estado_civil_groupby_pie.png?raw=true' style='display:block;float:none;margin-left:auto;margin-right:auto;width:70%'/>

**Será que o estado civil influencia no limite de crédito e nos valores de transações?**

```
SELECT AVG(valor_transacoes_12m) AS media_valor_transacoes, AVG(limite_credito) AS media_limite, estado_civil, sexo
FROM credito
WHERE estado_civil != 'na'
GROUP BY estado_civil, sexo
ORDER BY media_limite DESC;
```

<img src='https://github.com/mateus-miguel/projeto-credito-sql/blob/main/imagens/transacoes_limite_estado_civil_sexo.png?raw=true' style='display:block;float:none;margin-left:auto;margin-right:auto;width:70%'/>

Podemos ver que os casados são os que têm menores limite de crédito, tanto homens quanto mulheres, e gastam menos em transações. Já os divorciados são os que têm maior limite de crédito e gastam mais, similares aos solteiros.

<img src='https://github.com/mateus-miguel/projeto-credito-sql/blob/main/imagens/transacoes_limite_estado_civil_sexo_barra.png?raw=true' style='display:block;float:none;margin-left:auto;margin-right:auto;width:70%'/>

**Quais as características dos clientes que possuem os maiores créditos?**

```
SELECT MAX(limite_credito) AS max_limite_credito, escolaridade, tipo_cartao, sexo 
FROM credito
WHERE escolaridade != 'na'
GROUP BY escolaridade, tipo_cartao, sexo
ORDER BY max_limite_credito DESC;
```

A query acima permite analisar como escolaridade, sexo e tipo de cartão contribuem para o limite máximo de crédito dos clientes agrupados nestas categorias. A partir de seu resultado, observa-se que mesmo clientes sem educação formal e com o cartão tipo 'blue' mais utilizado têm os mais altos limites. Porém, aparentemente os clientes de sexo masculino costumam ter os maiores limites (topo da tabela), então o sexo pode ser um fator determinante.

<img src='https://github.com/mateus-miguel/projeto-credito-sql/blob/main/imagens/limite_credito_escolaridade_cartao_sexo_groupby.png?raw=true' style='display:block;float:none;margin-left:auto;margin-right:auto;width:70%'/>

**Apesar de não terem os maiores limites, será que as mulheres gastam mais?**

Usando a query

```
SELECT MAX(valor_transacoes_12m) AS max_valor, AVG(valor_transacoes_12m) AS media_valor, MIN(valor_transacoes_12m) AS min_valor, sexo
FROM credito
GROUP BY sexo;
```

<img src='https://github.com/mateus-miguel/projeto-credito-sql/blob/main/imagens/valor_transacoes_sexo_groupby.png?raw=true' style='display:block;float:none;margin-left:auto;margin-right:auto;width:70%'/>

De onde vemos que, apesar da diferença de limites, em relação a valores das transações os gastos de homens e mulheres são bem similares. A média de gastos das mulheres é levemente mais alto.

<img src='https://github.com/mateus-miguel/projeto-credito-sql/blob/main/imagens/valor_transacoes_sexo_groupby_barra.png?raw=true' style='display:block;float:none;margin-left:auto;margin-right:auto;width:70%'/>

**Qual efeito do sexo e faixa salarial sobre o limite do cartão e valores de transações?**

```
SELECT AVG(valor_transacoes_12m) AS media_valor_transacoes, AVG(limite_credito) AS media_limite, sexo, salario_anual
FROM credito
WHERE salario_anual != 'na'
GROUP BY sexo, salario_anual
ORDER BY media_valor_transacoes DESC;
```

<img src='https://github.com/mateus-miguel/projeto-credito-sql/blob/main/imagens/media_valor_limite_sexo_salario_groupby.png?raw=true' style='display:block;float:none;margin-left:auto;margin-right:auto;width:70%'/>

Podemos ver que os homens de maior faixa salarial têm uma média de limite de crédito maior, e aliado a isso gastam menos em transações. As mulheres têm dados limitados, talvez por ganharem apenas nas faixas salariais mais baixas neste conjunto de dados (menos de \\$40K e \\$40K - \\$60K).

<img src='https://github.com/mateus-miguel/projeto-credito-sql/blob/main/imagens/media_valor_limite_sexo_salario_groupby_barra.png?raw=true' style='display:block;float:none;margin-left:auto;margin-right:auto;width:80%'/>

## 4. Conclusões

Com base nas análises feitas, é possível extrair alguns *insights* valiosos:

* A maior parte da base de dados é formada por homens
* As idades dos clientes são semelhantes tanto para homens quanto para mulheres
* O maior grupo de clientes recebe na menor faixa de menos de \\$40K e a minoria ganha na faixa mais alta de \\$120K + 
* O tipo de cartão mais utilizado é o tipo blue
* Os homens têm os maiores limites de crédito, mas o limite não parece ser influenciado por escolaridade ou tipo de cartão
* Os gastos em transações são equivalentes para homens e mulheres
* Os homens das maiores faixas salariais têm os maiores limites de crédito e acabam gastando menos, em média, em transações. Logo, a faixa salarial tem impacto direto no limite e transações.
* Não existem clientes femininos ganhando acima de \\$60K por ano.
* A maioria dos clientes são casados, e em média os casados têm os menores limites de crédito assim como gastam menos em transações.